In [1]:
import numpy as np

arrs = []
with open('used_car_train_20200313.csv', 'r') as f:
    for line in f:
        arr = line.strip().split(' ')
        arrs.append(arr)

name = arrs[0]

train = np.array(arrs[1:])

In [2]:
for i in range(len(name)):
    print(name[i])
    if name[i].startswith('v'):
        continue
    values = list(set(train[:,i]))
    if len(values) > 10:
        print(values[0:10],'...')
    else:
        print(values)

SaleID
['126761', '52177', '12852', '104803', '112324', '149196', '107015', '34987', '61397', '67419'] ...
name
['126761', '52177', '12852', '104803', '178446', '34987', '76410', '148998', '167633', '22926'] ...
regDate
['20001111', '19960102', '20070904', '20100005', '19920803', '20130212', '19951111', '20120810', '20120503', '20010305'] ...
model
['', '98.0', '44.0', '163.0', '21.0', '212.0', '211.0', '220.0', '49.0', '56.0'] ...
brand
['0', '13', '29', '36', '8', '20', '32', '1', '31', '33'] ...
bodyType
['', '0.0', '5.0', '1.0', '2.0', '7.0', '3.0', '6.0', '4.0']
fuelType
['', '0.0', '5.0', '1.0', '2.0', '3.0', '6.0', '4.0']
gearbox
['', '1.0', '0.0']
power
['130', '76', '29', '400', '123', '1005', '249', '279', '224', '234'] ...
kilometer
['8.0', '5.0', '0.5', '10.0', '1.0', '2.0', '9.0', '7.0', '3.0', '6.0'] ...
notRepairedDamage
['-', '1.0', '0.0']
regionCode
['919', '400', '741', '1925', '2641', '274', '2145', '5301', '4913', '3858'] ...
seller
['0', '1']
offerType
['0']
creatD

In [8]:
class_feature = ['name', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'notRepairedDamage', 'seller', 'offerType']
result_feature = ['price']

continues_value = np.zeros((train.shape[0], train.shape[1]-len(class_feature)-len(result_feature)))

t = 0
for i in range(len(name)):
    if name[i] not in class_feature and name[i] not in result_feature:
        continues_value[:,t] = [float(v) for v in train[:,i]]
        t += 1

class_value = np.zeros((train.shape[0], len(class_feature))).astype(np.int64)

t = 0
for i in range(len(name)):
    
    if name[i] in class_feature:
        print(name[i])
        types = list(set(train[:,i]))
        print(len(types))
        convert = {}
        for tps in range(len(types)):
            convert[types[tps]] = tps
        class_value[:,t] = [convert[v] for v in train[:,i]]
        t += 1

result_value = np.zeros((train.shape[0], 1))

result_value = [float(v) for v in train[:,15]]

name
99662
model
249
brand
40
bodyType
9
fuelType
8
gearbox
3
notRepairedDamage
3
seller
2
offerType
1


In [17]:
import tensorflow as tf

inputs_0 = tf.keras.Input(shape=(continues_value.shape[1],))
inputs_1 = tf.keras.Input(shape=(1,))
inputs_2 = tf.keras.Input(shape=(1,))
inputs_3 = tf.keras.Input(shape=(1,))
inputs_4 = tf.keras.Input(shape=(1,))
inputs_5 = tf.keras.Input(shape=(1,))
inputs_6 = tf.keras.Input(shape=(1,))
inputs_7 = tf.keras.Input(shape=(1,))
inputs_8 = tf.keras.Input(shape=(1,))

emb_1 = tf.keras.layers.Embedding(99662,4)(inputs_1)
emb_2 = tf.keras.layers.Embedding(249,2)(inputs_2)
emb_3 = tf.keras.layers.Embedding(40,1)(inputs_3)
emb_4 = tf.keras.layers.Embedding(9,1)(inputs_4)
emb_5 = tf.keras.layers.Embedding(8,1)(inputs_5)
emb_6 = tf.keras.layers.Embedding(3,1)(inputs_6)
emb_7 = tf.keras.layers.Embedding(3,1)(inputs_7)
emb_8 = tf.keras.layers.Embedding(2,1)(inputs_8)

x = tf.keras.layers.concatenate([emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8])
x = tf.keras.backend.squeeze(x, axis=1)

x = tf.keras.layers.concatenate([inputs_0,x])
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(8, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dense(2, activation='relu')(x)
# x = tf.keras.layers.BatchNormalization()(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.models.Model(inputs=[inputs_0,inputs_1,inputs_2,
                                      inputs_3,inputs_4,inputs_5,inputs_6,inputs_7,inputs_8], 
                              outputs=output)

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae'])

In [16]:
history = model.fit([continues_value,class_value[:,0],class_value[:,1],class_value[:,2]
           ,class_value[:,3],class_value[:,4],class_value[:,5],class_value[:,6]
           ,class_value[:,7]],
         result_value, 
         epochs=100, 
         validation_split=0.1,
         batch_size=128)

Train on 135000 samples, validate on 15000 samples
Epoch 1/100
135000/135000 [==============================] - 7s 55us/sample - loss: 63476932.9308 - mean_absolute_error: 4754.7852 - val_loss: 62281057.9429 - val_mean_absolute_error: 4747.5757
Epoch 2/100
135000/135000 [==============================] - 7s 54us/sample - loss: 61168869.7033 - mean_absolute_error: 4651.9775 - val_loss: 57752245.7963 - val_mean_absolute_error: 4595.4175
Epoch 3/100
135000/135000 [==============================] - 7s 54us/sample - loss: 58885735.9479 - mean_absolute_error: 4544.1392 - val_loss: 56327297.5829 - val_mean_absolute_error: 4502.9824
Epoch 4/100
135000/135000 [==============================] - 7s 54us/sample - loss: 56534351.7672 - mean_absolute_error: 4433.9370 - val_loss: 54115493.6213 - val_mean_absolute_error: 4387.3188
Epoch 5/100
135000/135000 [==============================] - 7s 55us/sample - loss: 54136081.5012 - mean_absolute_error: 4319.3535 - val_loss: 55617335.4699 - val_mean_absol

135000/135000 [==============================] - 7s 54us/sample - loss: 3329671.1068 - mean_absolute_error: 962.1207 - val_loss: 4086279.6443 - val_mean_absolute_error: 857.6154
Epoch 44/100
135000/135000 [==============================] - 8s 59us/sample - loss: 3168374.5139 - mean_absolute_error: 948.6487 - val_loss: 4702660.5799 - val_mean_absolute_error: 955.9420
Epoch 45/100
135000/135000 [==============================] - 9s 67us/sample - loss: 3135718.8178 - mean_absolute_error: 941.3331 - val_loss: 4156555.1418 - val_mean_absolute_error: 832.8897
Epoch 46/100
135000/135000 [==============================] - 9s 70us/sample - loss: 3163450.2694 - mean_absolute_error: 946.8956 - val_loss: 7066633.1387 - val_mean_absolute_error: 936.0635
Epoch 47/100
135000/135000 [==============================] - 9s 69us/sample - loss: 2908931.1087 - mean_absolute_error: 902.9670 - val_loss: 23386113.3693 - val_mean_absolute_error: 1067.8920
Epoch 48/100
135000/135000 [============================

Epoch 86/100
135000/135000 [==============================] - 7s 54us/sample - loss: 1343512.6958 - mean_absolute_error: 754.1183 - val_loss: 5943281.3206 - val_mean_absolute_error: 977.9748
Epoch 87/100
135000/135000 [==============================] - 7s 54us/sample - loss: 1362386.0767 - mean_absolute_error: 751.1250 - val_loss: 7204835.4595 - val_mean_absolute_error: 903.7839
Epoch 88/100
135000/135000 [==============================] - 7s 55us/sample - loss: 1314150.2992 - mean_absolute_error: 749.3793 - val_loss: 6309822.1800 - val_mean_absolute_error: 932.8512
Epoch 89/100
135000/135000 [==============================] - 7s 54us/sample - loss: 1265774.0341 - mean_absolute_error: 736.9069 - val_loss: 6259195.2024 - val_mean_absolute_error: 789.8983
Epoch 90/100
135000/135000 [==============================] - 7s 55us/sample - loss: 1335040.5336 - mean_absolute_error: 741.3212 - val_loss: 7587728.9197 - val_mean_absolute_error: 837.6125
Epoch 91/100
135000/135000 [=================

In [18]:
history = model.fit([continues_value,class_value[:,0],class_value[:,1],class_value[:,2]
           ,class_value[:,3],class_value[:,4],class_value[:,5],class_value[:,6]
           ,class_value[:,7]],
         result_value, 
         epochs=100, 
         validation_split=0.1,
         batch_size=128)

Train on 135000 samples, validate on 15000 samples
Epoch 1/100
135000/135000 [==============================] - 8s 59us/sample - loss: 91261276.2515 - mean_absolute_error: 5916.4336 - val_loss: 88503212.4928 - val_mean_absolute_error: 5896.4512
Epoch 2/100
135000/135000 [==============================] - 7s 53us/sample - loss: 90114934.4313 - mean_absolute_error: 5878.4546 - val_loss: 87351532.8192 - val_mean_absolute_error: 5858.5474
Epoch 3/100
135000/135000 [==============================] - 7s 53us/sample - loss: 88201964.4297 - mean_absolute_error: 5810.3989 - val_loss: 85263564.6197 - val_mean_absolute_error: 5779.9795
Epoch 4/100
135000/135000 [==============================] - 7s 54us/sample - loss: 85682448.8770 - mean_absolute_error: 5717.4595 - val_loss: 81851086.4683 - val_mean_absolute_error: 5664.8916
Epoch 5/100
135000/135000 [==============================] - 7s 54us/sample - loss: 82691462.4204 - mean_absolute_error: 5603.1211 - val_loss: 79884828.2752 - val_mean_absol

135000/135000 [==============================] - 7s 52us/sample - loss: 1973795.0826 - mean_absolute_error: 786.4643 - val_loss: 2921275.5684 - val_mean_absolute_error: 732.3335
Epoch 44/100
135000/135000 [==============================] - 7s 52us/sample - loss: 1893715.4912 - mean_absolute_error: 789.1292 - val_loss: 3693544.7317 - val_mean_absolute_error: 847.3653
Epoch 45/100
135000/135000 [==============================] - 7s 52us/sample - loss: 1812653.3763 - mean_absolute_error: 778.7535 - val_loss: 4308263.7615 - val_mean_absolute_error: 1066.5795
Epoch 46/100
135000/135000 [==============================] - 8s 57us/sample - loss: 1695266.6989 - mean_absolute_error: 767.0173 - val_loss: 4468338.3115 - val_mean_absolute_error: 973.1097
Epoch 47/100
135000/135000 [==============================] - 7s 53us/sample - loss: 1713800.5211 - mean_absolute_error: 773.9920 - val_loss: 3309890.6069 - val_mean_absolute_error: 700.7095
Epoch 48/100
135000/135000 [=============================

Epoch 86/100
135000/135000 [==============================] - 7s 53us/sample - loss: 1084353.6485 - mean_absolute_error: 730.6561 - val_loss: 3835146.3613 - val_mean_absolute_error: 765.5203
Epoch 87/100
135000/135000 [==============================] - 7s 53us/sample - loss: 1070725.9143 - mean_absolute_error: 715.8992 - val_loss: 3805386.8450 - val_mean_absolute_error: 746.1825
Epoch 88/100
135000/135000 [==============================] - 7s 53us/sample - loss: 1059618.0334 - mean_absolute_error: 714.8325 - val_loss: 4305665.4539 - val_mean_absolute_error: 765.2752
Epoch 89/100
135000/135000 [==============================] - 7s 53us/sample - loss: 1093881.2586 - mean_absolute_error: 724.0847 - val_loss: 4509130.3574 - val_mean_absolute_error: 998.3058
Epoch 90/100
135000/135000 [==============================] - 7s 53us/sample - loss: 1082668.6562 - mean_absolute_error: 721.4303 - val_loss: 4024384.0662 - val_mean_absolute_error: 796.4847
Epoch 91/100
135000/135000 [=================